In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from numpy import load
import os
import cv2
import random
from matplotlib import pyplot

In [0]:
model_save_path = '/content/drive/My Drive/summer2winter_yosemite/savedModels/'

import zipfile
with zipfile.ZipFile('/content/drive/My Drive/summer2winter_yosemite.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/')

In [0]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-32jpofbl
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-32jpofbl
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=173f54a837c9cca66cc79133e0e49247c176eef7973eda2a67a381cf0b36b7aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-gg53o3s8/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,Flatten,Input,BatchNormalization,AveragePooling2D,LeakyReLU,ReLU,Concatenate,Conv2DTranspose,Add
from keras import optimizers
from keras.optimizers import Adam
from keras.models import Model
from keras.losses import mean_squared_error
from keras.initializers import RandomNormal
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

Using TensorFlow backend.


dataset_trainA = []
dataset_trainB = []
dataset_testA = []
dataset_testB = []
for i in trainA_files:
  dataset_trainA.append(img_to_array(load_img(trainA_path + i, target_size=(256,256))))
for i in trainB_files:
  dataset_trainB.append(img_to_array(load_img(trainB_path + i, target_size=(256,256))))
for i in testA_files:
  dataset_testA.append(img_to_array(load_img(testA_path + i, target_size=(256,256))))
for i in testB_files:
  dataset_testB.append(img_to_array(load_img(testB_path + i, target_size=(256,256))))  

dataset_trainA = np.asarray(dataset_trainA)
dataset_trainB = np.asarray(dataset_trainB)
dataset_testA = np.asarray(dataset_testA)
dataset_testB = np.asarray(dataset_testB)
from numpy import savez_compressed
filename = 'drive/My Drive/summer2winter_yosemite/summer2winter_yosemite.npz'
savez_compressed(filename, dataset_trainA, dataset_trainB,dataset_testA,dataset_testB)
print('Saved dataset: ', filename)
trainA_path = '/content/drive/My Drive/summer2winter_yosemite/trainA/'
trainB_path = '/content/drive/My Drive/summer2winter_yosemite/trainB/'
testA_path = '/content/drive/My Drive/summer2winter_yosemite/testA/'
testB_path = '/content/drive/My Drive/summer2winter_yosemite/testB/'
import os
trainA_files = os.listdir(trainA_path)
trainB_files = os.listdir(trainB_path)
testA_files = os.listdir(testA_path)
testB_files = os.listdir(testB_path)

In [0]:
import tensorflow as tf
from keras.engine.topology import Layer
from keras.engine import InputSpec

In [0]:
def patchGanBlock(input_img,filt_size,numFilters,numStrides,weight_init):
  patch_img = Conv2D(filters = numFilters,kernel_size = (filt_size,filt_size),strides = numStrides,padding = 'same',kernel_initializer=weight_init)(input_img)
  patch_img = InstanceNormalization(axis=-1)(patch_img)
  patch_img = LeakyReLU(alpha=0.2)(patch_img)
  
  return patch_img

In [0]:
def discriminatorBlock(input_shape):
  weight_init = RandomNormal(stddev=0.02)

  input_img = Input(shape=input_shape)
  layer_out = Conv2D(filters = 64,kernel_size = (4,4),strides = 2,padding = 'same',kernel_initializer=weight_init)(input_img)
  layer_out = LeakyReLU(alpha=0.2)(layer_out)

  layer_out = patchGanBlock(layer_out,4,128,2,weight_init)
  layer_out = patchGanBlock(layer_out,4,256,2,weight_init)
  layer_out = patchGanBlock(layer_out,4,512,2,weight_init)

  layer_out = Conv2D(filters = 512,kernel_size = (4,4),padding = 'same',kernel_initializer=weight_init)(layer_out)
  layer_out = InstanceNormalization(axis=-1)(layer_out)
  layer_out = LeakyReLU(alpha=0.2)(layer_out)  

  layer_out = Conv2D(filters = 1,kernel_size = (4,4),padding = 'same',kernel_initializer=weight_init)(layer_out)

  model = Model(inputs=input_img,outputs=layer_out)
  model.compile(optimizer=Adam(lr=0.0002, beta_1=0.5),loss = 'mse',loss_weights=[0.5],metrics = ['accuracy'])

  return model

In [0]:
input_shape = (256,256,3)
model_discriminator = discriminatorBlock(input_shape)
model_discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      3136      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_1 (In (None, 64, 64, 128)       256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 256)       5245

In [0]:
def convBlock(input_img,filt_size,numFilters,numStrides,weight_init,paddingType,applyActivation):
  conv_img = Conv2D(filters = numFilters,kernel_size = (filt_size,filt_size),strides = numStrides,padding = paddingType,kernel_initializer=weight_init)(input_img)
  conv_img = InstanceNormalization(axis=-1)(conv_img)
  if applyActivation:
    conv_img = Activation('relu')(conv_img)
  return conv_img

In [0]:
def residual_convBlock(input_img,numFilters):
  weight_init = RandomNormal(stddev=0.02)
  conv_img = convBlock(input_img,3,numFilters,1,weight_init,'same',True)
  
  #conv_img = ReflectionPadding2D(padding=(1,1))(conv_img)  
  #conv_img = Conv2D(filters = numFilters,kernel_size = (3,3),strides = 1,padding = 'same',kernel_initializer=weight_init)(conv_img)  
  #conv_img = InstanceNormalization(axis=-1)(conv_img)  
  conv_img = convBlock(input_img,3,numFilters,1,weight_init,'same',False)
  
  conv_img = Concatenate()([conv_img, input_img])	
  return conv_img

In [0]:
def unet_block(input_img,filt_size,numFilters,numStrides,weight_init):
  conv_img = Conv2DTranspose(filters = numFilters,kernel_size = (filt_size,filt_size),strides = numStrides,padding = 'same',kernel_initializer=weight_init)(input_img)
  conv_img = InstanceNormalization(axis=-1)(conv_img)
  conv_img = Activation('relu')(conv_img)
  return conv_img  

In [0]:
def generatorBlock(input_shape,numResnet):
  weight_init = RandomNormal(stddev=0.02)

  input_img = Input(shape=input_shape)

  conv_out = convBlock(input_img,7,64,1,weight_init,'same',True)
  conv_out = convBlock(conv_out,3,128,2,weight_init,'same',True)
  conv_out = convBlock(conv_out,3,256,2,weight_init,'same',True)

  for i in range(0,numResnet):
    conv_out = residual_convBlock(conv_out,256)
  
  conv_out = unet_block(conv_out,3,128,2,weight_init)
  conv_out = unet_block(conv_out,3,64,2,weight_init)
  
  conv_out = convBlock(conv_out,7,3,1,weight_init,'same',False)
  output_img = Activation('tanh')(conv_out)
  model = Model(inputs=input_img,outputs=output_img)
  
  return model

In [0]:
input_shape = (256,256,3)
model_generator = generatorBlock(input_shape,9)
model_generator.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 256, 256, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
instance_normalization_5 (Insta (None, 256, 256, 64) 128         conv2d_7[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           instance_normalization_5[0][0]   
____________________________________________________________________________________________

In [0]:
input_shape = (256,256,3)

model_gen_a = generatorBlock(input_shape,9)
model_gen_b = generatorBlock(input_shape,9)

model_disc_a = discriminatorBlock(input_shape)
model_disc_b = discriminatorBlock(input_shape)

In [0]:
from keras.models import load_model

cust = {'InstanceNormalization': InstanceNormalization}
model_gen_a = load_model(model_save_path + 'model_gen_a.h5', cust)
model_gen_b = load_model(model_save_path + 'model_gen_b.h5', cust)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
model_disc_a = load_model(model_save_path + 'model_disc_a.h5', cust)
model_disc_b = load_model(model_save_path + 'model_disc_b.h5', cust)

In [0]:
def build_collated_model(gen_1,gen_2,disc,input_shape):
  gen_1.trainable = True
  gen_2.trainable = False
  disc.trainable = False

  input_img = Input(shape=input_shape)
  
  gen_1_out = gen_1(input_img)
  disc_out_1 = disc(gen_1_out)

  #identity mapping
  input_img_id = Input(shape=input_shape)
  gen_1_id = gen_1(input_img_id)  

  #forward cycle
  out_forward = gen_2(gen_1_out)

  #backward cycle
  gen_2_out = gen_2(input_img_id)
  out_backward = gen_1(gen_2_out)

  collated_model = Model(inputs=[input_img,input_img_id],outputs = [disc_out_1,gen_1_id,out_forward,out_backward])
  collated_model.compile(optimizer=Adam(lr=0.0002, beta_1=0.5),loss = ['mse','mae','mae','mae'],loss_weights = [1,5,10,10])
  return collated_model

In [0]:
input_shape = (256,256,3)

gan_model_a_To_b_To_a = build_collated_model(model_gen_a,model_gen_b,model_disc_b,input_shape)
gan_model_b_To_a_To_b = build_collated_model(model_gen_b,model_gen_a,model_disc_a,input_shape)

In [0]:
def generateRealSamples(dataset,patchSize,n_samples):
  fileId = np.random.randint(0,dataset.shape[0],n_samples)
  X = []
  
  for i in fileId:    
    imVal = dataset[i]
    assert imVal.shape == (256, 256, 3)
    imVal = imVal.astype(np.float32)
    imVal = (imVal-127.5) / 127.5    
    X.append(imVal)
  
  y = np.ones((n_samples,patchSize,patchSize,1))
  return np.asarray(X),y


In [0]:
def generateFakeSamples(modelBlock,dataset,patchSize):
  #print(dataset.shape)
  X_fake = modelBlock.predict(dataset)
  y_fake = np.zeros((dataset.shape[0],patchSize,patchSize,1))
  return X_fake,y_fake

In [0]:
def updatePool(poolImages,maxSize,images):
  selectedImage = list()
  for i in images:
    if len(poolImages) < maxSize:
      poolImages.append(i)
      selectedImage.append(i)
    elif random.random() < 0.5:
      selectedImage.append(i)
    else:
      idx = np.random.randint(0,len(poolImages))
      selectedImage.append(poolImages[idx])
      poolImages[idx] = i
  return np.asarray(selectedImage)

In [0]:
def train_model(dataset_trainA,dataset_trainB,dataset_testA,dataset_testB,
                model_gen_a,model_gen_b,gan_model_b_To_a_To_b,gan_model_a_To_b_To_a,
                model_disc_a,model_disc_b,model_save_path):
  poolImagesA = []
  poolImagesB = []
  maxSize = 50
  numEpochs = 100
  batchSize = 1
  patchSize = model_disc_a.output_shape[1]
  bat_per_epoch = int(dataset_trainA.shape[0]/batchSize)
  num_steps = int(numEpochs*bat_per_epoch)
  accuracy_A,accuracy_B = 0,0
  for i in range(120639,num_steps):
    X_real_A, y_real_A = generateRealSamples(dataset_trainA,patchSize,batchSize)
    X_real_B, y_real_B = generateRealSamples(dataset_trainB,patchSize,batchSize)

    X_fake_A,y_fake_A = generateFakeSamples(model_gen_b,X_real_B,patchSize)
    X_fake_B,y_fake_B = generateFakeSamples(model_gen_a,X_real_A,patchSize)

    X_fake_A = updatePool(poolImagesA,maxSize,X_fake_A)
    X_fake_B = updatePool(poolImagesB,maxSize,X_fake_B)

    b_a_b_loss,_,_,_,_ = gan_model_b_To_a_To_b.train_on_batch([X_real_B,X_real_A],[y_real_A,X_real_A,X_real_B,X_real_A])

    disc_a_loss1,_ = model_disc_a.train_on_batch(X_real_A,y_real_A)   
    disc_a_loss2,accuracy_A = model_disc_a.train_on_batch(X_fake_A,y_fake_A)
    
    a_b_a_loss,_,_,_,_ = gan_model_a_To_b_To_a.train_on_batch([X_real_A,X_real_B],[y_real_B,X_real_B,X_real_A,X_real_B])

    disc_b_loss1,_ = model_disc_b.train_on_batch(X_real_B,y_real_B)
    disc_b_loss2,accuracy_B = model_disc_b.train_on_batch(X_fake_B,y_fake_B)

    print('>%d, discA[%.3f,%.3f] discB[%.3f,%.3f] g[%.3f,%.3f]' % 
          (i+1, disc_a_loss1,disc_a_loss2, disc_b_loss1,disc_b_loss2, a_b_a_loss,b_a_b_loss))
    
    if (i+1) % (bat_per_epoch * 1) == 0:
      gan_model_b_To_a_To_b.save(model_save_path + 'gan_model_b_To_a_To_b.h5')
      gan_model_a_To_b_To_a.save(model_save_path + 'gan_model_a_To_b_To_a.h5')
      model_gen_a.save(model_save_path + 'model_gen_a.h5')
      model_gen_b.save(model_save_path + 'model_gen_b.h5')
      model_disc_a.save(model_save_path + 'model_disc_a.h5')
      model_disc_b.save(model_save_path + 'model_disc_b.h5')

      X_real_A_test, y_real_A_test = generateRealSamples(dataset_testA,patchSize,batchSize)
      X_real_B_test, y_real_B_test = generateRealSamples(dataset_testB,patchSize,batchSize)
      X_fake_B_test,y_fake_B_test = generateFakeSamples(model_gen_a,X_real_A_test,patchSize)
      X_real_A_test = (X_real_A_test + 1) / 2.0
      X_fake_B_test = (X_fake_B_test + 1) / 2.0
	    # plot real images
      fig = pyplot.figure(figsize=(10, 10))
      ax = fig.add_subplot(2, 1, 1)
      ax.axis('off')
      ax.imshow(np.squeeze(X_real_A_test))
	    
      # plot translated image	    
      ax = fig.add_subplot(2, 1, 2)
      ax.axis('off')
      ax.imshow(np.squeeze(X_fake_B_test))
	    # save plot to file
      filename1 = '%s_generated_plot_%06d.png' % ('Summer_To_Winter', (i))
      pyplot.savefig(model_save_path + filename1)
      pyplot.close()      

In [0]:
dataset = load('drive/My Drive/summer2winter_yosemite/summer2winter_yosemite.npz')
dataset_trainA = dataset['arr_0'] 
dataset_trainB = dataset['arr_1']
dataset_testA = dataset['arr_2'] 
dataset_testB = dataset['arr_3']

In [0]:
train_model(dataset_trainA,dataset_trainB,dataset_testA,dataset_testB,
                model_gen_a,model_gen_b,gan_model_b_To_a_To_b,gan_model_a_To_b_To_a,
                model_disc_a,model_disc_b,model_save_path)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>120640, discA[0.002,0.010] discB[0.002,0.004] g[3.088,2.819]
>120641, discA[0.002,0.014] discB[0.030,0.059] g[2.314,2.810]
>120642, discA[0.003,0.011] discB[0.006,0.007] g[3.089,3.208]
>120643, discA[0.002,0.005] discB[0.004,0.009] g[2.843,2.741]
>120644, discA[0.024,0.006] discB[0.002,0.006] g[2.948,2.824]
>120645, discA[0.007,0.003] discB[0.021,0.007] g[2.353,2.743]
>120646, discA[0.003,0.008] discB[0.002,0.029] g[3.590,3.376]
>120647, discA[0.010,0.015] discB[0.003,0.021] g[2.973,2.570]
>120648, discA[0.004,0.004] discB[0.004,0.007] g[2.382,2.642]
>120649, discA[0.001,0.007] discB[0.006,0.017] g[2.306,2.705]
>120650, discA[0.001,0.002] discB[0.003,0.008] g[2.957,2.774]
>120651, discA[0.002,0.004] discB[0.007,0.005] g[2.599,2.784]
>120652, discA[0.004,0.006] discB[0.026,0.005] g[2.639,2.482]
>120653, discA[0.002,0.007] discB[0.002,0.004] g[2.643,2.823]
>120654, discA[0.002,0.004] discB[0.010,0.003] g[2.184,2.277]
>120655, discA[0.001,0.004] discB[0.014,0.004] g[2.860,3.817]
>120656,

In [0]:
for i in range(0,dataset_testB.shape[0]):  
  X = []  
  imVal = dataset_testB[i]
  assert imVal.shape == (256, 256, 3)
  imVal = imVal.astype(np.float32)
  imVal = (imVal-127.5) / 127.5    
  X.append(imVal)
  X_real_A_test = np.asarray(X)

  X_fake_B_test,y_fake_B_test = generateFakeSamples(model_gen_b,X_real_A_test,16)
  X_real_A_test = (X_real_A_test + 1) / 2.0
  X_fake_B_test = (X_fake_B_test + 1) / 2.0
  # plot real images
  fig = pyplot.figure(figsize=(10, 10))
  ax = fig.add_subplot(2, 1, 1)
  ax.axis('off')
  ax.imshow(np.squeeze(X_real_A_test))
	    
      # plot translated image	    
  ax = fig.add_subplot(2, 1, 2)
  ax.axis('off')
  ax.imshow(np.squeeze(X_fake_B_test))
  filename1 = '%s_generated_plot_%06d.png' % ('Winter_to_Summer_', (i))
  pyplot.savefig(model_save_path + 'testPics/Winter2Summer/' + filename1)
  pyplot.close()       